<a href="https://colab.research.google.com/github/MnCSSJ4x/Content-based-recommendation/blob/main/TextToNetflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Netflix Movies Reccomendation System 
Goal is to suggest a movie if a user liked a specific movie from the list of movies on Netflix 

In [154]:
import numpy as np 
import pandas as pd 
import re
import tqdm 
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [155]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Dataset from: https://www.kaggle.com/datasets/narayan63/netflix-popular-movies-dataset

In [156]:
df = pd.read_csv('n_movies.csv')

In [157]:
df

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"
...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,After an experimental gene therapy turns them ...,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',...","3,130"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a com...,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis...","970,067"
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,898"
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,Two brothers follow their father's footsteps a...,"['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ...","439,601"


In [158]:
# df = df.dropna(subset=['title','genre','description','stars'])
df = df.dropna()

In [159]:
df = df.drop(df[df['genre'] == ''].index)
df = df.drop(df[df['description'] == ''].index)
df = df.drop(df[df['stars'] == ''].index)
df

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"
...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,After an experimental gene therapy turns them ...,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',...","3,130"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a com...,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melis...","970,067"
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,898"
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,Two brothers follow their father's footsteps a...,"['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim ...","439,601"


In [160]:
def preprocess(text):
  text = re.sub("\'", "", text)
  text = re.sub("[^a-zA-Z]"," ",text)
  text = ' '.join(text.split())
  text = text.lower()
  return text

df['preproc_description'] = df['description'].apply(lambda x: preprocess(x))
df['preproc_genre'] = df['genre'].apply(lambda x: preprocess(x))
df.head()

,title,year,certificate,duration,genre,rating,description,stars,votes,preproc_description,preproc_genre
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031",decades after their all valley karate tourname...,action comedy drama
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885",follows the political rivalries and romance of...,biography drama history
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384",the trials and tribulations of criminal lawyer...,crime drama
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773",when a psychiatrist shelters a mysterious cult...,drama horror mystery
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413",a street kid trying to survive in a technology...,animation action adventure


In [161]:
df['stars_combined'] = df['stars'].apply(lambda x: ''.join(x))
df['stars_combined'] = df['stars_combined'].apply(lambda x: preprocess(x))
df['text'] = df["preproc_description"] + " " + df["stars_combined"] + " " + df["preproc_genre"]
df['text'] = df['text'].apply(lambda x:x.lower())

In [162]:
df.head(10)

,title,year,certificate,duration,genre,rating,description,stars,votes,preproc_description,preproc_genre,stars_combined,text
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031",decades after their all valley karate tourname...,action comedy drama,ralph macchio william zabka courtney henggeler...,decades after their all valley karate tourname...
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885",follows the political rivalries and romance of...,biography drama history,claire foy olivia colman imelda staunton matt ...,follows the political rivalries and romance of...
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384",the trials and tribulations of criminal lawyer...,crime drama,bob odenkirk rhea seehorn jonathan banks patri...,the trials and tribulations of criminal lawyer...
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773",when a psychiatrist shelters a mysterious cult...,drama horror mystery,emily deschanel sam jaeger gerardo celasco mad...,when a psychiatrist shelters a mysterious cult...
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413",a street kid trying to survive in a technology...,animation action adventure,zach aguilar kenichiro ohashi emi lo aoi y ki,a street kid trying to survive in a technology...
5,The Sandman,(2022– ),TV-MA,45 min,"Drama, Fantasy, Horror",7.8,Upon escaping after decades of imprisonment by...,"['Tom Sturridge, ', 'Boyd Holbrook, ', 'Patton...","116,358",upon escaping after decades of imprisonment by...,drama fantasy horror,tom sturridge boyd holbrook patton oswalt vivi...,upon escaping after decades of imprisonment by...
6,Rick and Morty,(2013– ),TV-MA,23 min,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"['Justin Roiland, ', 'Chris Parnell, ', 'Spenc...","502,160",an animated series that follows the exploits o...,animation adventure comedy,justin roiland chris parnell spencer grammer s...,an animated series that follows the exploits o...
7,Breaking Bad,(2008–2013),TV-MA,49 min,"Crime, Drama, Thriller",9.5,A high school chemistry teacher diagnosed with...,"['Bryan Cranston, ', 'Aaron Paul, ', 'Anna Gun...","1,831,340",a high school chemistry teacher diagnosed with...,crime drama thriller,bryan cranston aaron paul anna gunn betsy brandt,a high school chemistry teacher diagnosed with...
8,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,After an experimental gene therapy turns them ...,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',...","3,123",after an experimental gene therapy turns them ...,action adventure drama,morgan taylor campbell italia ricci rhianna ja...,after an experimental gene therapy turns them ...
9,Blonde,(2022),NC-17,166 min,"Biography, Drama, Mystery",6.2,A fictionalized chronicle of the inner life of...,"['Andrew Dominik', '| ', ' Stars:', 'Ana de...",935,a fictionalized chronicle of the inner life of...,biography drama mystery,andrew dominik stars ana de armas lucy devito ...,a fictionalized chronicle of the inner life of...


In [163]:
ls = []

def prompt(summary):
    ls.append(summary)
    for i in (df["text"]):
        ls.append(i)

In [164]:
model = SentenceTransformer('nli-distilroberta-base-v2')

In [165]:
prompt("A show with Bryan Cranston")

In [166]:
embedding = model.encode(ls)
embedding.shape

(5755, 768)

In [ ]:
sim = cosine_similarity([embedding[0]], embedding[1:]).flatten()
sim = list(enumerate(sim))
sim.sort(key=lambda x: x[1])
sim.reverse()
sim

In [168]:
recommended = sim[0:5]
for i in recommended:
    print(df.iloc[i[0]])

title                                 Standup and Away! with Brian Regan
year                                                            (2018– )
certificate                                                        TV-PG
duration                                                          24 min
genre                                                             Comedy
rating                                                               5.7
description            Bryan Regan blends his trademark observational...
stars                  ['Beth Triffon, ', 'Brian Regan, ', 'Kari Cole...
votes                                                                402
preproc_description    bryan regan blends his trademark observational...
preproc_genre                                                     comedy
stars_combined          beth triffon brian regan kari coleman matt riedy
text                   bryan regan blends his trademark observational...
Name: 6127, dtype: object
title                    